# TP 1 - Partie 1 : Transport Optimal Entropique

## Objectifs

Dans ce notebook, vous allez :
1. Comprendre le lien entre OT entropique et le problème de Schrödinger
2. Implémenter l'algorithme de Sinkhorn avec la bibliothèque POT
3. Visualiser les couplages optimaux entre distributions
4. Explorer l'effet du paramètre de régularisation entropique $\varepsilon$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ot
from scipy.spatial.distance import cdist
import sys
sys.path.append('../src')
from simulation import simulate_sde

# Configuration matplotlib
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## 1. Rappels théoriques

### Transport Optimal classique

Étant données deux distributions de probabilité $\mu$ et $\nu$ sur $\mathbb{R}^d$, le problème de transport optimal cherche le couplage $\pi$ qui minimise le coût de transport :

$$
\min_{\pi \in \Pi(\mu, \nu)} \mathbb{E}_{\pi}[c(X, Y)]
$$

où $c(x, y) = |x - y|^2$ est le coût quadratique.

### Transport Optimal Entropique

Pour régulariser le problème et le rendre plus stable numériquement, on ajoute un terme d'entropie :

$$
\gamma^*_{\varepsilon} = \arg\min_{\pi \in \Pi(\mu, \nu)} \left\{ \mathbb{E}_{\pi}[c(X, Y)] + \varepsilon H(\pi) \right\}
$$

où $H(\pi) = -\int \pi(x, y) \log \pi(x, y) dx dy$ est l'entropie de Shannon.

### Lien avec le problème de Schrödinger

Le transport optimal entropique peut s'écrire comme :

$$
\min_{\pi \in \Pi(\mu, \nu)} H\left(\pi \, \Big| \, e^{-\frac{|X-Y|^2}{\varepsilon}}\right)
$$

Cela correspond au problème de Schrödinger : trouver le processus stochastique le plus proche (au sens de l'entropie relative) du mouvement brownien, qui relie $\mu$ à $\nu$.

**Pour l'inférence de trajectoires** : Si $\varepsilon = \sigma \Delta t$ (où $\sigma$ est la diffusion de notre EDS), alors $\gamma^*_{\varepsilon}(x, \cdot)$ correspond au noyau de transition optimal d'une EDS à dérive gradient !

## 2. Chargement des données simulées

On charge les données que vous avez générées dans le notebook `00_introduction`.

In [ ]:
# Paramètres de simulation
n_particles = 1000
dim = 2
t0, t1 = 0.0, 1.0
dt = 1e-3
sigma = 0.5
snapshot_times = np.linspace(t0, t1, 6)

# Simulation avec le potentiel complexe
snapshots = simulate_sde(
    n_particles=n_particles,
    dim=dim,
    t0=t0,
    t1=t1,
    dt=dt,
    sigma=sigma,
    snapshot_times=snapshot_times,
    potential_type='complex',
    seed=42
)

print(f"Nombre de snapshots: {len(snapshots)}")
print(f"Temps des snapshots: {list(snapshots.keys())}")
print(f"Forme de chaque snapshot: {snapshots[t0].shape}")

### Visualisation des distributions temporelles

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, (t, X) in enumerate(snapshots.items()):
    ax = axes[idx]
    ax.scatter(X[:, 0], X[:, 1], alpha=0.5, s=10)
    ax.set_xlabel('$x_0$')
    ax.set_ylabel('$x_1$')
    ax.set_title(f'$t = {t:.2f}$')
    ax.axis('equal')
    ax.set_xlim(-5, 5)
    ax.set_ylim(-4, 4)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Transport Optimal Entropique avec POT

### 3.1 Comprendre l'algorithme de Sinkhorn

L'algorithme de Sinkhorn résout le problème d'OT entropique par une procédure itérative :

$$
\gamma^*_{\varepsilon} = \text{diag}(u) K \text{diag}(v)
$$

où $K_{ij} = e^{-c_{ij}/\varepsilon}$ est le noyau de Gibbs et $(u, v)$ sont les vecteurs de scaling obtenus par itération.

### 3.2 Première application : couplage entre deux snapshots consécutifs

In [ ]:
# Prenons deux snapshots consécutifs
times = sorted(snapshots.keys())
t_source = times[0]
t_target = times[1]
delta_t = t_target - t_source

X_source = snapshots[t_source]
X_target = snapshots[t_target]

n_source = len(X_source)
n_target = len(X_target)

print(f"Source: t={t_source:.2f}, {n_source} particules")
print(f"Target: t={t_target:.2f}, {n_target} particules")
print(f"Δt = {delta_t:.2f}")

### 3.3 Calcul de la matrice de coût

Pour l'OT, on utilise le coût quadratique $c(x, y) = |x - y|^2$.

In [ ]:
# TODO: Calculer la matrice de coût C entre X_source et X_target
# Hint: utiliser cdist avec metric='sqeuclidean'
C = # VOTRE CODE ICI

print(f"Forme de la matrice de coût: {C.shape}")
print(f"Coût min: {C.min():.4f}")
print(f"Coût max: {C.max():.4f}")
print(f"Coût moyen: {C.mean():.4f}")

### 3.4 Calcul du couplage optimal avec Sinkhorn

**Question théorique** : Quel devrait être le bon choix de $\varepsilon$ d'après le cours ?

<details>
<summary>Réponse</summary>
D'après le cours, pour que l'OT entropique corresponde au problème de Schrödinger associé à notre EDS, on doit choisir :
$$\varepsilon = \sigma \Delta t$$
où $\sigma$ est le coefficient de diffusion de notre EDS.
</details>

In [ ]:
# Distributions uniformes (empiriques)
a = np.ones(n_source) / n_source
b = np.ones(n_target) / n_target

# TODO: Calculer le paramètre de régularisation théorique epsilon_theory
epsilon_theory = # VOTRE CODE ICI
print(f"ε théorique = σ × Δt = {sigma} × {delta_t:.2f} = {epsilon_theory:.4f}")

# TODO: Calculer le couplage optimal avec ot.sinkhorn
# Hint: ot.sinkhorn(a, b, C, reg=epsilon_theory, numItermax=1000, stopThr=1e-9)
gamma_opt = # VOTRE CODE ICI

print(f"\nForme du couplage optimal: {gamma_opt.shape}")
print(f"Somme du couplage: {gamma_opt.sum():.6f} (devrait être ≈ 1)")
print(f"Coût du transport: {(gamma_opt * C).sum():.4f}")

### 3.5 Visualisation du couplage optimal

On peut visualiser le couplage de plusieurs façons :
1. Matrice de couplage (heatmap)
2. Correspondances entre particules
3. Transport plan

In [ ]:
# Visualisation 1 : Matrice de couplage (sous-échantillonnée pour la lisibilité)
n_display = 100
plt.figure(figsize=(8, 6))
plt.imshow(gamma_opt[:n_display, :n_display] * n_source * n_target, 
           cmap='viridis', interpolation='nearest')
plt.colorbar(label='Probabilité de couplage')
plt.xlabel('Particules cibles')
plt.ylabel('Particules sources')
plt.title(f'Matrice de couplage optimal (ε={epsilon_theory:.4f})\n(100 premières particules)')
plt.tight_layout()
plt.show()

In [ ]:
# Visualisation 2 : Plan de transport (on trace quelques connexions)
n_display = 50

fig, ax = plt.subplots(figsize=(10, 8))

# Tracer les distributions
ax.scatter(X_source[:, 0], X_source[:, 1], c='blue', alpha=0.5, s=30, label=f't={t_source:.2f}')
ax.scatter(X_target[:, 0], X_target[:, 1], c='red', alpha=0.5, s=30, label=f't={t_target:.2f}')

# Pour les n_display premières particules, tracer les connexions principales
for i in range(min(n_display, n_source)):
    # Trouver les k meilleures correspondances
    k = 3
    top_k = np.argsort(gamma_opt[i, :])[-k:]
    
    for j in top_k:
        weight = gamma_opt[i, j] * n_source
        if weight > 0.01:
            ax.plot([X_source[i, 0], X_target[j, 0]], 
                   [X_source[i, 1], X_target[j, 1]], 
                   'k-', alpha=min(weight, 1.0), linewidth=0.5)

ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_1$')
ax.set_title(f'Plan de transport optimal (ε={epsilon_theory:.4f})')
ax.legend()
ax.axis('equal')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Exercice : Effet du paramètre de régularisation ε

### Question

Que se passe-t-il quand on fait varier $\varepsilon$ ?
- Quand $\varepsilon \to 0$ : on se rapproche du transport optimal classique (couplage sparse)
- Quand $\varepsilon \to \infty$ : on se rapproche du couplage indépendant $\mu \otimes \nu$ (couplage diffus)

### À faire

Calculez le couplage optimal pour différentes valeurs de $\varepsilon$ et observez :
1. La structure de la matrice de couplage
2. Le coût du transport
3. La "diffusivité" du plan de transport

In [ ]:
# Liste de valeurs d'epsilon à tester
epsilons = [
    epsilon_theory / 10,
    epsilon_theory / 2,
    epsilon_theory,
    epsilon_theory * 2,
    epsilon_theory * 10
]

# TODO: Calculer les couplages et les coûts pour chaque epsilon
couplings = []
costs = []

for eps in epsilons:
    # Calculer le couplage optimal
    gamma = # VOTRE CODE ICI
    
    # Calculer le coût total
    cost = # VOTRE CODE ICI
    
    couplings.append(gamma)
    costs.append(cost)
    
    print(f"ε = {eps:.6f}, Coût = {cost:.4f}")

In [ ]:
# Visualisation des matrices de couplage pour différents epsilon
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

n_display = 100

for idx, (eps, gamma) in enumerate(zip(epsilons, couplings)):
    ax = axes[idx]
    im = ax.imshow(gamma[:n_display, :n_display] * n_source * n_target, 
                   cmap='viridis', interpolation='nearest', vmin=0, vmax=0.1)
    ax.set_title(f'ε = {eps:.6f}\nCoût = {costs[idx]:.4f}')
    ax.set_xlabel('Particules cibles')
    ax.set_ylabel('Particules sources')
    plt.colorbar(im, ax=ax)

axes[-1].axis('off')

plt.suptitle('Effet du paramètre de régularisation ε sur le couplage', fontsize=14, y=1.00)
plt.tight_layout()
plt.show()

### Analyse des résultats

**Questions à discuter** :

1. Comment la structure de la matrice de couplage évolue-t-elle avec $\varepsilon$ ?
2. Que se passe-t-il avec le coût de transport quand $\varepsilon$ augmente ?
3. Visuellement, quel $\varepsilon$ semble donner le meilleur compromis ?
4. Pourquoi le choix $\varepsilon = \sigma \Delta t$ a-t-il un sens physique ?

## 5. Mesure de la sparsité du couplage

On peut quantifier la "diffusivité" du couplage en mesurant son entropie :

$$
H(\gamma) = -\sum_{i,j} \gamma_{ij} \log \gamma_{ij}
$$

Plus l'entropie est faible, plus le couplage est concentré (sparse).

In [ ]:
def compute_entropy(gamma):
    """Calcule l'entropie de Shannon d'un couplage."""
    gamma_pos = gamma[gamma > 1e-20]
    return -np.sum(gamma_pos * np.log(gamma_pos))

# Calculer les entropies
entropies = [compute_entropy(gamma) for gamma in couplings]

# Tracer coût vs entropie
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Coût en fonction de epsilon
ax1.plot(epsilons, costs, 'o-', linewidth=2, markersize=8)
ax1.axvline(epsilon_theory, color='red', linestyle='--', label='$\\epsilon$ théorique')
ax1.set_xscale('log')
ax1.set_xlabel('$\\epsilon$')
ax1.set_ylabel('Coût de transport')
ax1.set_title('Coût vs régularisation')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Entropie en fonction de epsilon
ax2.plot(epsilons, entropies, 'o-', linewidth=2, markersize=8, color='green')
ax2.axvline(epsilon_theory, color='red', linestyle='--', label='$\\epsilon$ théorique')
ax2.set_xscale('log')
ax2.set_xlabel('$\\epsilon$')
ax2.set_ylabel('Entropie H(γ)')
ax2.set_title('Entropie vs régularisation')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nRésumé :")
print("="*50)
for eps, cost, entropy in zip(epsilons, costs, entropies):
    print(f"ε = {eps:.6f} | Coût = {cost:.4f} | Entropie = {entropy:.2f}")

## 6. Conclusion

Dans ce notebook, vous avez :

✅ Compris le lien entre OT entropique et le problème de Schrödinger  
✅ Implémenté l'algorithme de Sinkhorn avec POT  
✅ Visualisé les couplages optimaux  
✅ Exploré l'effet du paramètre de régularisation $\varepsilon$  

**Points clés à retenir** :

1. L'OT entropique régularise le problème classique et le rend plus stable numériquement
2. Le choix $\varepsilon = \sigma \Delta t$ a une justification théorique via le problème de Schrödinger
3. Le paramètre $\varepsilon$ contrôle le compromis entre fidélité aux données (coût faible) et régularité (entropie élevée)

**Prochaine étape** : Dans le notebook suivant (`02_trajectory_inference.ipynb`), nous allons utiliser ces couplages pour reconstruire des trajectoires complètes et calculer l'interpolation de McCann.